In [1]:
import re
import os
import numpy as np
from progress.bar import Bar
from tqdm import tqdm
import time

In [2]:

def writeRadius(r1,r2):
    lpMesh = open("loop.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()

    reg1 = r"(%r1)"
    reg2 = r"(%r2)"

    sub1 = str(r1)
    sub2 = str(r2)

    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)

    try:
        lpTemp = open("loopTemp.geo", "x")
    except:
        os.remove("loopTemp.geo")
        lpTemp = open("loopTemp.geo","x")

    lpTemp.write(loopMesh)
    lpTemp.close()


    # write surface mesh
    lpMesh = open("loopFIELD.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()
    reg1 = r"(%r1)"
    reg2 = r"(%r2)"
    sub1 = str(r1)
    sub2 = str(r2)
    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)
    try:
        lpTemp = open("loopFIELDTemp.geo", "x")
    except:
        os.remove("loopFIELDTemp.geo")
        lpTemp = open("loopFIELDTemp.geo","x")
    lpTemp.write(loopMesh)
    lpTemp.close()

def runGMSH():
    print(f"{os.getpid()}: Meshing geometry\n")
    # os.system("gmsh loopFIELDTemp.geo -nopopup -0 -v 1")
    # os.system("gmsh loopTemp.geo -nopopup -0 -v 1")
    os.popen("gmsh loopFIELDTemp.geo -nopopup -0 -v 1").read()
    os.popen("gmsh loopTemp.geo -nopopup -0 -v 1").read()
    try:
        os.system("rm *.geo_unrolled")
        os.remove("loopFIELDTemp.geo")
        os.remove("loopTemp.geo")
    except:
        print("nope")
    print(f"{os.getpid()}: Done meshing\n")


def runTTH():
    print(f"{os.getpid()}: Running tth settings_washer.json >/dev/null 2>&1")
    os.system("tth settings_washer.json > tthlog.txt")
    print(f"{os.getpid()}: Done\n")


def getCurrent():
    iFile = open("./output/i.txt")
    it = iFile.read()
    iFile.close()
    reg = r"fone fone .+ (\d*\.\d*e.\d*)"
    i_s = re.findall(reg,it,re.MULTILINE)[0]
    return i_s
    
def runNoiseEx(b_path,current):
    path = "/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex"
    print(f"{os.getpid()}: Running "+path+" "+b_path+" "+current)
    v = os.popen(path+" "+b_path+" "+current).read()
    print(f"{os.getpid()}: Done\n")
    v = v.split("\n")
    return v

def getElementsAndNodeCount():
    iFile = open("./loop.msh")
    it = iFile.read()
    iFile.close()
    regNumNodes = r"\$Nodes\n(.*)\n"
    numNodes = re.findall(regNumNodes,it,re.MULTILINE)[0]
    regNumElem = r"\$Elements\n(.*)\n"
    numElem = re.findall(regNumElem,it,re.MULTILINE)[0]

    return numElem, numNodes



### Triangles:

In [3]:
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/")

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1s = np.linspace(30e-6, 40e-6, 30)
r2s = np.linspace(5e-6, 0.15e-6, 30)


try:
    os.remove("triangles.csv")
except:
    pass

res = open("triangles.csv","w")
res.write("R,D,Analytic,Numerical,Analytic Area,Numerical Area,Mesh Time,TTH Time,nex Time,Num Elements,Num Nodes\n")

for r1,r2 in tqdm(zip(r1s,r2s)):
    writeRadius(r1, r2)
    start = time.time()
    runGMSH()
    end = time.time()
    meshTime = end - start

    numElem , numNodes = getElementsAndNodeCount()

    start = time.time()
    runTTH()
    end = time.time()
    tthTime = end - start
    i = getCurrent()
    print(i)
    start = time.time()
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk",i)
    end = time.time()
    nexTime = end - start
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area},{meshTime},{tthTime},{nexTime},{numElem},{numNodes}\n")



res.close()




0it [00:00, ?it/s]

151669: Meshing geometry

151669: Done meshing

151669: Running tth settings_washer.json >/dev/null 2>&1


Unknown device "/dev/sda": No such file or directory


151669: Done

2.1731997539e-01
151669: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 2.1731997539e-01


1it [00:17, 17.33s/it]

...

151669: Done

151669: Meshing geometry

151669: Done meshing

151669: Running tth settings_washer.json >/dev/null 2>&1


Unknown device "/dev/sda": No such file or directory


151669: Done

3.2976186566e-02
151669: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 3.2976186566e-02


30it [10:33, 21.10s/it]

151669: Done



### Tetras:

In [ ]:
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/")

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1s = np.linspace(30e-6, 40e-6, 30)
r2s = np.linspace(5e-6, 0.15e-6, 30)


try:
    os.remove("tetras.csv")
except:
    pass

res = open("tetras.csv","w")
res.write("R,D,Analytic,Numerical,Analytic Area,Numerical Area,Mesh Time,TTH Time,nex Time,Num Elements,Num Nodes\n")

for r1,r2 in tqdm(zip(r1s,r2s)):
    writeRadius(r1, r2)
    start = time.time()
    runGMSH()
    end = time.time()
    meshTime = end - start

    numElem , numNodes = getElementsAndNodeCount()

    start = time.time()
    runTTH()
    end = time.time()
    tthTime = end - start
    i = getCurrent()
    print(i)
    start = time.time()
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk",i)
    end = time.time()
    nexTime = end - start
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area},{meshTime},{tthTime},{nexTime},{numElem},{numNodes}\n")



res.close()

os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/")

In [ ]:
writeRadius(15e-6,0.2e-6)